<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/03_Analisis_Fase_2A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-

"""
================================================================================
FASE 2A - EVALUACIÓN CUANTITATIVA Y ANÁLISIS GEOMÉTRICO
================================================================================
Trabajo Fin de Máster: Evaluación Comparativa de Técnicas de Segmentación
en Fotografía de Retrato

Autor: Jesús Lévano Lévano
Universidad: Universidad Oberta de Catalunya (UOC)
Tutor: Miguel Alejandro Ponce Proaño
Fecha: Noviembre 2025

DESCRIPCIÓN:
Fase 2A del análisis comparativo de modelos de segmentación. Calcula métricas
cuantitativas exhaustivas comparando máscaras predichas por 5 modelos contra
ground truth anotado manualmente, con análisis geométrico avanzado usando
Shapely y análisis textural usando Mahotas.

DEPENDENCIAS DE DATOS:
- Requiere Fase 1 completada (índice maestro + configuraciones)
- Utiliza máscaras NPZ de modelos en ubicaciones originales
- Utiliza ground truth desde anotaciones CVAT
- Utiliza características fotográficas del Notebook 00

MODELOS EVALUADOS:
1. Mask2Former (Swin-Large/Tiny, ADE20K/COCO) - 19 configuraciones
2. OneFormer (task-conditioned) - 36 configuraciones
3. SAM 2.0 (con/sin prompts) - 44 configuraciones
4. YOLOv8-seg (nano a xlarge) - 20 configuraciones
5. BodyPix (MobileNetV1) - 8 configuraciones
Total: 127 configuraciones únicas

DATASET:
- 20 fotografías de retrato manualmente anotadas
- Resoluciones variables (típicamente 6016×4016 px)
- Ground truth binario en formato NPZ
- Total evaluaciones: 20 fotos × 127 configs = 2,540 combinaciones

MÉTRICAS CALCULADAS:

A. Métricas Clásicas de Segmentación (5 métricas):
   - IoU (Intersection over Union) - Métrica estándar
   - Dice Coefficient - Solapamiento normalizado
   - Precision - Tasa de verdaderos positivos
   - Recall - Cobertura del ground truth
   - F1-Score - Media armónica precision-recall

B. Métricas Geométricas con Shapely (25 métricas):

   B1. Geometría Básica:
       - Área (pixels)
       - Perímetro (pixels)
       - Bounding box (xmin, ymin, xmax, ymax)
       - Centroide (x, y)
       - Aspect ratio (ancho/alto)
       - Orientación (grados)

   B2. Forma Avanzada:
       - Convex hull (envolvente convexa)
       - Solidity (área / área_convex_hull)
       - Compacidad (4π×área / perímetro²)
       - Circularity
       - Elongation (ratio ejes)
       - Rectangularity (área / área_bbox)

   B3. Contorno:
       - Boundary IoU (solo bordes)
       - Hausdorff Distance (error máximo)
       - Chamfer Distance (error promedio)
       - Smoothness (varianza curvatura)
       - Número de vértices

   B4. Posición Compositiva:
       - Distancia a centro imagen
       - Zona regla de tercios (1-9)
       - Recortes por bordes (%)
       - Espacio negativo

   B5. Fragmentación:
       - Número componentes desconectadas
       - Área componente principal
       - Ratio fragmentación

   B6. Comparación vs Ground Truth:
       - Desplazamiento centroide (Euclidean)
       - Diferencia orientación (grados)
       - Ratio solidity (pred/GT)
       - Diferencia compacidad
       - Symmetric difference area

C. Métricas Texturales con Mahotas (30 métricas):

   C1. Texturas Haralick - Región Interior (13 features):
       - Angular Second Moment
       - Contrast
       - Correlation
       - Sum of Squares Variance
       - Inverse Difference Moment
       - Sum Average
       - Sum Variance
       - Sum Entropy
       - Entropy
       - Difference Variance
       - Difference Entropy
       - Information Measure Correlation 1
       - Information Measure Correlation 2

   C2. Texturas Haralick - Región Borde (13 features):
       - Mismas features calculadas en borde de 5px
       - Evalúa calidad de separación persona-fondo

   C3. Estadísticas Básicas (4 métricas):
       - Media, std, min, max de intensidad

TOTAL MÉTRICAS POR CONFIGURACIÓN: ~60 métricas

PROCESAMIENTO:
- Incremental con sistema de checkpoint resumible
- Guardado automático después de cada fotografía procesada
- Gestión optimizada de memoria para Google Colab free tier
- Manejo robusto de configuraciones faltantes
- Logging detallado para trazabilidad

ESTRUCTURA DE SALIDA:
/TFM/3_Analisis/fase2_evaluacion/
├── metricas_individuales/
│   ├── _DSC0036_metricas_completas.json
│   ├── _DSC0592_metricas_completas.json
│   └── ... (20 archivos JSON)
│
├── metricas_agregadas/
│   ├── todas_metricas.csv                  # Tabla plana: 2540 filas × ~65 columnas
│   ├── ranking_configuraciones.csv         # TOP-10 configs ordenadas por IoU
│   ├── estadisticas_por_modelo.json        # Media, std, min, max por modelo
│   ├── configuraciones_omitidas.json       # Registro de omisiones
│   └── datos_visualizaciones.json          # Datos pre-formateados para plots
│
├── checkpoint_fase2a.json                  # Estado de procesamiento

REFERENCIAS METODOLÓGICAS:
- Lin, T.Y., et al. (2014). Microsoft COCO: Common Objects in Context
- Kirillov, A., et al. (2023). Segment Anything
- Shapely Documentation (2024). Geometric Operations
- Haralick, R.M., et al. (1973). Textural Features for Image Classification
- Gonzalez, R.C. & Woods, R.E. (2018). Digital Image Processing (4th ed.)
================================================================================
"""

'\n================================================================================\nFASE 2A - EVALUACIÓN CUANTITATIVA Y ANÁLISIS GEOMÉTRICO\n================================================================================\nTrabajo Fin de Máster: Evaluación Comparativa de Técnicas de Segmentación\nen Fotografía de Retrato\n\nAutor: Jesús Lévano Lévano\nUniversidad: Universidad Oberta de Catalunya (UOC)\nTutor: Miguel Alejandro Ponce Proaño\nFecha: Noviembre 2025\n\nDESCRIPCIÓN:\nFase 2A del análisis comparativo de modelos de segmentación. Calcula métricas\ncuantitativas exhaustivas comparando máscaras predichas por 5 modelos contra\nground truth anotado manualmente, con análisis geométrico avanzado usando\nShapely y análisis textural usando Mahotas.\n\nDEPENDENCIAS DE DATOS:\n- Requiere Fase 1 completada (índice maestro + configuraciones)\n- Utiliza máscaras NPZ de modelos en ubicaciones originales\n- Utiliza ground truth desde anotaciones CVAT\n- Utiliza características fotográficas d

In [ ]:
!pip install -q shapely mahotas scikit-image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# =============================================================================
# IMPORTACIÓN DE LIBRERÍAS
# =============================================================================

# Librerías estándar de Python
import json
import logging
import sys
from dataclasses import dataclass, field
from datetime import datetime
from enum import Enum
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Any, Set

# Librerías de procesamiento numérico y datos
import numpy as np
import pandas as pd

# Librerías de procesamiento de imágenes
import cv2
import mahotas
from PIL import Image
from scipy.ndimage import distance_transform_edt
from scipy.spatial.distance import directed_hausdorff
from skimage import measure

# Shapely para análisis geométrico
from shapely.geometry import Polygon, MultiPolygon, Point
from shapely.validation import make_valid
from shapely.ops import unary_union

In [ ]:
# =============================================================================
# DEFINICIÓN DE ESTRUCTURAS DE DATOS
# =============================================================================

class NivelLogging(Enum):
    """
    Enumeración de niveles de logging para control de verbosidad.

    Niveles:
    - MINIMAL: Solo resumen por fotografía
    - NORMAL: Progreso con detalle moderado (recomendado)
    - DETAILED: Información detallada por configuración
    - DEBUG: Máximo detalle para debugging
    """
    MINIMAL = "minimal"
    NORMAL = "normal"
    DETAILED = "detailed"
    DEBUG = "debug"

In [ ]:
@dataclass
class ConfiguracionFase2A:
    """
    Dataclass para configuración global de Fase 2A.

    Attributes:
        ruta_base_tfm: Directorio raíz del proyecto TFM
        max_dimension_mahotas: Redimensión para Mahotas (optimización)
        ancho_borde_px: Ancho del borde para análisis textural
        usar_checkpoint: Habilita sistema de checkpoint resumible
        guardar_json_individuales: Guardar JSON por fotografía
        guardar_csv_consolidado: Guardar CSV con todas las métricas
        guardar_estadisticas: Calcular estadísticas por modelo
        nivel_logging: Nivel de verbosidad del logging
    """
    ruta_base_tfm: Path
    max_dimension_mahotas: int = 1024
    ancho_borde_px: int = 5
    usar_checkpoint: bool = True
    guardar_json_individuales: bool = True
    guardar_csv_consolidado: bool = True
    guardar_estadisticas: bool = True
    nivel_logging: NivelLogging = NivelLogging.NORMAL

    @property
    def ruta_fase1(self) -> Path:
        """Ruta al directorio de resultados de Fase 1."""
        return self.ruta_base_tfm / "3_Analisis" / "fase1_integracion"

    @property
    def ruta_salida(self) -> Path:
        """Ruta al directorio de salida de Fase 2A."""
        return self.ruta_base_tfm / "3_Analisis" / "fase2_evaluacion"

    @property
    def ruta_ground_truth(self) -> Path:
        """Ruta al directorio de ground truth."""
        return self.ruta_base_tfm / "0_Imagenes_CVAT" / "ground_truth_masks"

    @property
    def ruta_caracteristicas(self) -> Path:
        """Ruta al directorio de características fotográficas."""
        return self.ruta_base_tfm / "1_Caracteristicas" / "json"

    def crear_directorios(self) -> None:
        """Crea la estructura de directorios de salida."""
        (self.ruta_salida / "metricas_individuales").mkdir(parents=True, exist_ok=True)
        (self.ruta_salida / "metricas_agregadas").mkdir(parents=True, exist_ok=True)


@dataclass
class CheckpointFase2A:
    """
    Dataclass para gestión de checkpoint de procesamiento.

    Permite reanudar el procesamiento desde el punto de interrupción,
    evitando reprocesar fotografías ya completadas.

    Attributes:
        fotos_procesadas: Conjunto de códigos de fotografías completadas
        fotos_pendientes: Lista de códigos de fotografías por procesar
        configuraciones_totales: Número total de configuraciones
        ultima_actualizacion: Timestamp de última actualización
    """
    fotos_procesadas: Set[str] = field(default_factory=set)
    fotos_pendientes: List[str] = field(default_factory=list)
    configuraciones_totales: int = 0
    ultima_actualizacion: Optional[str] = None

    def to_dict(self) -> Dict[str, Any]:
        """Serializa checkpoint a diccionario JSON-compatible."""
        return {
            "fotos_procesadas": list(self.fotos_procesadas),
            "fotos_pendientes": self.fotos_pendientes,
            "configuraciones_totales": self.configuraciones_totales,
            "ultima_actualizacion": self.ultima_actualizacion
        }

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'CheckpointFase2A':
        """Reconstruye checkpoint desde diccionario."""
        return cls(
            fotos_procesadas=set(data.get("fotos_procesadas", [])),
            fotos_pendientes=data.get("fotos_pendientes", []),
            configuraciones_totales=data.get("configuraciones_totales", 0),
            ultima_actualizacion=data.get("ultima_actualizacion")
        )

In [ ]:
# =============================================================================
# FUNCIONES AUXILIARES
# =============================================================================

def configurar_logging(nivel: NivelLogging, nombre_modulo: str = "Fase2A") -> logging.Logger:
    """
    Configuración del sistema de logging con formato académico.

    Args:
        nivel: Nivel de logging deseado
        nombre_modulo: Nombre del módulo para identificación

    Returns:
        Logger configurado
    """
    logger = logging.getLogger(nombre_modulo)
    logger.setLevel(logging.DEBUG)

    if logger.handlers:
        logger.handlers.clear()

    handler = logging.StreamHandler(sys.stdout)

    if nivel == NivelLogging.MINIMAL:
        handler.setLevel(logging.WARNING)
    elif nivel == NivelLogging.NORMAL:
        handler.setLevel(logging.INFO)
    elif nivel == NivelLogging.DETAILED:
        handler.setLevel(logging.INFO)
    else:
        handler.setLevel(logging.DEBUG)

    formatter = logging.Formatter(
        fmt='[%(asctime)s] %(levelname)-8s | %(message)s',
        datefmt='%H:%M:%S'
    )

    handler.setFormatter(formatter)
    logger.addHandler(handler)

    return logger


def guardar_json_seguro(datos: Any, ruta_archivo: Path, logger: logging.Logger) -> bool:
    """
    Guarda datos en formato JSON con manejo robusto de errores.

    Incluye conversión automática de tipos NumPy y sets a formatos JSON-compatibles.

    Args:
        datos: Datos a serializar
        ruta_archivo: Path donde guardar el JSON
        logger: Logger para registrar operaciones

    Returns:
        True si guardado exitoso, False en caso contrario
    """
    try:
        class NumpyEncoder(json.JSONEncoder):
            """Encoder personalizado para tipos NumPy y estructuras complejas."""
            def default(self, obj):
                if isinstance(obj, np.integer):
                    return int(obj)
                if isinstance(obj, np.floating):
                    return float(obj)
                if isinstance(obj, np.ndarray):
                    return obj.tolist()
                if isinstance(obj, set):
                    return list(obj)
                if isinstance(obj, Path):
                    return str(obj)
                return super().default(obj)

        ruta_archivo.parent.mkdir(parents=True, exist_ok=True)

        with open(ruta_archivo, 'w', encoding='utf-8') as f:
            json.dump(datos, f, indent=2, ensure_ascii=False, cls=NumpyEncoder)

        return True

    except Exception as e:
        logger.error(f"Error guardando JSON en {ruta_archivo}: {e}")
        return False

# ==============================================================================
# MODIFICACIÓN FASE 2A - SOPORTE SAM2 Y SAM2 PROMPTS
# ==============================================================================
#
# Este archivo contiene:
# 1. Nueva función cargar_mascara_npz (reemplaza líneas 388-512)
# 2. Nueva función auxiliar _cargar_sam2_prompts_mejor_mascara
# 3. Instrucciones para modificar el orquestador
#
# ==============================================================================

def _cargar_sam2_prompts_mejor_mascara(
    ruta_npz: Path,
    logger: Optional[logging.Logger] = None
) -> Tuple[Optional[np.ndarray], Optional[Dict[str, Any]]]:
    """
    Carga la mejor máscara de SAM2 Prompts buscando todos los archivos persona_*.

    SAM2 Prompts genera múltiples archivos por foto (persona_0, persona_1, etc.).
    Esta función busca todos los archivos correspondientes y selecciona el mejor
    según el criterio: mayor person_confidence, con desempate por mayor área.

    Args:
        ruta_npz: Path al archivo NPZ de referencia (puede ser cualquier persona_N)
        logger: Logger opcional para debugging

    Returns:
        Tuple de (mascara_bool, metadata) con la mejor máscara seleccionada
    """
    directorio = ruta_npz.parent
    nombre_archivo = ruta_npz.name

    # Extraer foto_id del nombre: {foto_id}_persona_{N}_{timestamp}.npz
    partes = nombre_archivo.split('_persona_')
    if len(partes) != 2:
        if logger:
            logger.error(f"Formato de nombre SAM2 Prompts no reconocido: {nombre_archivo}")
        return None, None

    foto_id = partes[0]

    # Buscar todos los archivos persona_* para esta foto en este directorio
    patron = f"{foto_id}_persona_*.npz"
    archivos_persona = sorted(directorio.glob(patron))

    if not archivos_persona:
        if logger:
            logger.warning(f"No se encontraron archivos persona_* para {foto_id} en {directorio}")
        return None, None

    if logger:
        logger.debug(f"SAM2 Prompts: encontrados {len(archivos_persona)} archivos para {foto_id}")

    # Cargar todos y seleccionar el mejor
    candidatos = []

    for archivo in archivos_persona:
        try:
            with np.load(archivo, allow_pickle=False) as data:
                if 'mascara' not in data:
                    continue

                nombre = archivo.name
                num_persona = int(nombre.split('_persona_')[1].split('_')[0])

                candidatos.append({
                    'archivo': archivo,
                    'num_persona': num_persona,
                    'mascara': data['mascara'].copy(),
                    'area': int(data.get('area', np.sum(data['mascara']))),
                    'score': float(data.get('score', 0)),
                    'person_confidence': float(data.get('person_confidence', 0)),
                    'criterios_cumplidos': int(data.get('criterios_cumplidos', 0))
                })
        except Exception as e:
            if logger:
                logger.warning(f"Error cargando {archivo}: {e}")
            continue

    if not candidatos:
        if logger:
            logger.error(f"No se pudieron cargar candidatos válidos para {foto_id}")
        return None, None

    # Selección: Mayor person_confidence, desempate por mayor área
    mejor = max(candidatos, key=lambda x: (x['person_confidence'], x['area']))

    # Construir metadata
    metadata = {
        'tipo_seleccion': 'sam2_prompts',
        'num_candidatos': len(candidatos),
        'persona_seleccionada': mejor['num_persona'],
        'archivo_seleccionado': mejor['archivo'].name,
        'confidence_seleccionada': mejor['person_confidence'],
        'area_seleccionada': mejor['area'],
        'score_sam_seleccionado': mejor['score'],
        'criterios_cumplidos': mejor['criterios_cumplidos'],
        'criterio': 'mayor_confidence_desempate_area',
        'resumen_candidatos': [
            {
                'persona': c['num_persona'],
                'confidence': c['person_confidence'],
                'area': c['area']
            }
            for c in sorted(candidatos, key=lambda x: x['num_persona'])
        ]
    }

    if logger:
        logger.debug(
            f"SAM2 Prompts: seleccionada persona_{mejor['num_persona']} de {len(candidatos)} "
            f"(conf={mejor['person_confidence']:.2f}, área={mejor['area']:,}px)"
        )

    mascara_bool = mejor['mascara'].astype(bool)

    return mascara_bool, metadata


# ------------------------------------------------------------------------------
# PARTE 2: FUNCIÓN PRINCIPAL
# ------------------------------------------------------------------------------

def cargar_mascara_npz(
    ruta_npz: Path,
    umbral: Optional[float] = None,
    logger: Optional[logging.Logger] = None
) -> Tuple[Optional[np.ndarray], Optional[Dict[str, Any]]]:
    """
    Carga máscara desde archivo NPZ con manejo de múltiples estructuras.

    Soporta seis estructuras de almacenamiento utilizadas por los diferentes
    modelos evaluados en el TFM:

    1. Unificada inglés: clave 'person_mask' (OneFormer, YOLO, BodyPix)
    2. Unificada español: clave 'mascara' (Mask2Former)
    3. SAM2 automático: clave 'mascaras_binarias' con múltiples candidatas
    4. SAM2 Prompts: múltiples archivos persona_* (búsqueda dinámica)
    5. Ground Truth CVAT: clave 'masks' con array 3D [1, H, W]
    6. Por umbral: claves 'mask_X.XX' (estructura legacy)

    Notas metodológicas:
        - SAM2 automático: Se selecciona la máscara de mayor área entre las que
          tienen máxima confianza (person_confidence).
        - SAM2 Prompts: Se buscan todos los archivos persona_* de la misma foto
          y configuración, seleccionando el de mayor person_confidence.

    Args:
        ruta_npz: Path al archivo NPZ que contiene la máscara de segmentación.
        umbral: Umbral específico para modelos con estructura por-umbral.
        logger: Logger opcional para mensajes de debugging y errores.

    Returns:
        Tuple de (mascara_bool, metadata):
            - mascara_bool: Array booleano 2D con la máscara, o None si error
            - metadata: Dict con info de selección (SAM2/SAM2 Prompts) o None
    """
    if not ruta_npz.exists():
        if logger:
            logger.warning(f"Archivo NPZ no encontrado: {ruta_npz}")
        return None, None

    try:
        # =============================================================
        # Detección especial: SAM2 Prompts (múltiples archivos persona_*)
        # =============================================================
        nombre_archivo = ruta_npz.name

        if '_persona_' in nombre_archivo:
            return _cargar_sam2_prompts_mejor_mascara(ruta_npz, logger)

        # =============================================================
        # Carga estándar para otros modelos
        # =============================================================
        with np.load(ruta_npz, allow_pickle=False) as data:
            claves_disponibles = list(data.keys())

            if logger:
                logger.debug(f"Claves en NPZ: {claves_disponibles}")

            mascara = None
            metadata = None

            # ---------------------------------------------------------
            # Caso 1: Estructura unificada inglés (OneFormer, YOLO, BodyPix)
            # ---------------------------------------------------------
            if 'person_mask' in claves_disponibles:
                mascara = data['person_mask']
                if logger:
                    logger.debug(f"Cargada máscara 'person_mask', shape: {mascara.shape}")

            # ---------------------------------------------------------
            # Caso 2: Estructura unificada español (Mask2Former)
            # ---------------------------------------------------------
            elif 'mascara' in claves_disponibles:
                mascara = data['mascara']
                if logger:
                    logger.debug(f"Cargada máscara 'mascara', shape: {mascara.shape}")

            # ---------------------------------------------------------
            # Caso 3: Estructura SAM2 automático (múltiples máscaras)
            # ---------------------------------------------------------
            elif 'mascaras_binarias' in claves_disponibles:
                mascaras = data['mascaras_binarias']
                num_mascaras = int(data.get('num_mascaras', 0))

                if num_mascaras == 0 or len(mascaras) == 0:
                    if logger:
                        logger.warning(f"SAM2 NPZ sin máscaras válidas: {ruta_npz}")
                    return None, None

                confidences = data['person_confidences']
                areas = data['areas']

                # Seleccionar: mayor área entre las de máxima confianza
                max_conf = np.max(confidences)
                indices_top = np.where(confidences == max_conf)[0]
                mejor_idx = indices_top[np.argmax(areas[indices_top])]

                mascara = mascaras[mejor_idx]

                metadata = {
                    'tipo_seleccion': 'sam2_automatico',
                    'num_candidatos': int(num_mascaras),
                    'indice_seleccionado': int(mejor_idx),
                    'confidence_seleccionada': float(confidences[mejor_idx]),
                    'area_seleccionada': int(areas[mejor_idx]),
                    'max_confidence_disponible': float(max_conf),
                    'criterio': 'mayor_area_en_maxima_confianza'
                }

                if logger:
                    logger.debug(
                        f"SAM2: seleccionada máscara {mejor_idx} de {num_mascaras} "
                        f"(conf={max_conf:.2f}, área={areas[mejor_idx]:,}px)"
                    )

            # ---------------------------------------------------------
            # Caso 4: Ground Truth CVAT (estructura con 'masks')
            # ---------------------------------------------------------
            elif 'masks' in claves_disponibles:
                mascara = data['masks']
                if logger:
                    logger.debug(f"Cargada máscara Ground Truth CVAT, shape: {mascara.shape}")

                if mascara.ndim == 3:
                    mascara = mascara[0]
                    if logger:
                        logger.debug(f"Reducida dimensión 3D a 2D: {mascara.shape}")

            # ---------------------------------------------------------
            # Caso 5: Estructura por-umbral (legacy)
            # ---------------------------------------------------------
            else:
                claves_mascara = [k for k in claves_disponibles if k.startswith('mask_')]

                if not claves_mascara:
                    if logger:
                        logger.error(
                            f"Estructura NPZ no reconocida en {ruta_npz}. "
                            f"Claves disponibles: {claves_disponibles}. "
                            f"Se esperaba: 'person_mask', 'mascara', 'mascaras_binarias', 'masks' o 'mask_X.XX'"
                        )
                    return None, None

                if umbral is not None:
                    clave_objetivo = f"mask_{umbral:.2f}"

                    if clave_objetivo not in claves_disponibles:
                        if logger:
                            logger.warning(
                                f"Umbral {umbral:.2f} no encontrado en {ruta_npz}. "
                                f"Disponibles: {claves_mascara}"
                            )
                        return None, None

                    mascara = data[clave_objetivo]
                    if logger:
                        logger.debug(f"Cargada máscara por umbral '{clave_objetivo}', shape: {mascara.shape}")

                else:
                    if logger:
                        logger.error(
                            f"Estructura por-umbral detectada pero no se especificó 'umbral'. "
                            f"Claves disponibles: {claves_mascara}"
                        )
                    return None, None

            # ---------------------------------------------------------
            # Validación final
            # ---------------------------------------------------------
            if mascara is None:
                if logger:
                    logger.error(f"No se pudo extraer máscara de {ruta_npz}")
                return None, None

            if mascara.ndim != 2:
                if logger:
                    logger.error(
                        f"Máscara con dimensiones inválidas en {ruta_npz}: "
                        f"shape={mascara.shape}, se esperaba 2D"
                    )
                return None, None

            mascara_bool = mascara.astype(bool)

            return mascara_bool, metadata

    except Exception as e:
        if logger:
            logger.error(f"Error cargando NPZ {ruta_npz}: {e}")
        return None, None

def mascara_a_shapely(mascara: np.ndarray, logger: Optional[logging.Logger] = None) -> Optional[Polygon | MultiPolygon]:
    """
    Convierte máscara binaria a polígonos Shapely.

    Utiliza find_contours de scikit-image para detectar contornos y los
    convierte en geometrías Shapely válidas. Maneja múltiples componentes
    desconectadas y aplica validación geométrica.

    Args:
        mascara: Array 2D booleano con la máscara
        logger: Logger opcional para debugging

    Returns:
        Polygon o MultiPolygon válido, o None si error

    Notes:
        - Invierte coordenadas Y debido a convención imagen vs Shapely
        - Filtra polígonos con menos de 3 vértices
        - Aplica make_valid() para corregir auto-intersecciones
        - Une polígonos múltiples usando unary_union para evitar errores
    """
    try:
        # Asegurar que la máscara es binaria
        mascara_binaria = mascara.astype(np.uint8)

        # Encontrar contornos
        contornos = measure.find_contours(mascara_binaria, level=0.5)

        if len(contornos) == 0:
            if logger:
                logger.debug("No se encontraron contornos en la máscara")
            return None

        poligonos_validos = []

        for contorno in contornos:
            # Invertir coordenadas Y (convención imagen vs Shapely)
            coordenadas = [(x, mascara.shape[0] - y) for y, x in contorno]

            # Filtrar contornos con muy pocos puntos
            if len(coordenadas) < 3:
                continue

            try:
                # Crear polígono
                poligono = Polygon(coordenadas)

                # Validar y corregir si es necesario
                if not poligono.is_valid:
                    poligono = make_valid(poligono)

                # Verificar que sigue siendo válido después de make_valid
                if poligono.is_valid and not poligono.is_empty:
                    # Si make_valid devolvió un MultiPolygon, extraer polígonos individuales
                    if isinstance(poligono, MultiPolygon):
                        for poly in poligono.geoms:
                            if poly.is_valid and not poly.is_empty:
                                poligonos_validos.append(poly)
                    else:
                        poligonos_validos.append(poligono)

            except Exception as e_poly:
                if logger:
                    logger.debug(f"Error creando polígono individual: {e_poly}")
                continue

        if len(poligonos_validos) == 0:
            if logger:
                logger.debug("No se generaron polígonos válidos")
            return None

        # Si hay un solo polígono, retornarlo directamente
        if len(poligonos_validos) == 1:
            return poligonos_validos[0]

        # Si hay múltiples polígonos, usar unary_union para combinarlos correctamente
        try:
            geometria_unida = unary_union(poligonos_validos)

            # Validar el resultado final
            if not geometria_unida.is_valid:
                geometria_unida = make_valid(geometria_unida)

            if geometria_unida.is_empty:
                if logger:
                    logger.warning("La unión de polígonos resultó vacía")
                return None

            return geometria_unida

        except Exception as e_union:
            if logger:
                logger.error(f"Error en unary_union: {e_union}")

            # Fallback: retornar MultiPolygon directamente
            try:
                return MultiPolygon(poligonos_validos)
            except Exception as e_mp:
                if logger:
                    logger.error(f"Error creando MultiPolygon: {e_mp}")
                return None

    except Exception as e:
        if logger:
            logger.error(f"Error convirtiendo máscara a Shapely: {e}")
        return None

In [ ]:
# =============================================================================
# CALCULADORAS DE MÉTRICAS
# =============================================================================

def calcular_metricas_clasicas(mascara_pred: np.ndarray, mascara_gt: np.ndarray) -> Dict[str, float]:
    """
    Calcula métricas clásicas de segmentación (IoU, Dice, Precision, Recall, F1).

    Si las máscaras tienen dimensiones diferentes, redimensiona la predicción
    al tamaño del ground truth usando interpolación nearest-neighbor para
    preservar naturaleza binaria.

    Args:
        mascara_pred: Máscara predicha (H1, W1) binaria
        mascara_gt: Máscara ground truth (H2, W2) binaria

    Returns:
        Diccionario con 5 métricas: iou, dice, precision, recall, f1_score

    Notes:
        El redimensionamiento es solo para comparación píxel a píxel.
        Las métricas geométricas (Shapely) se calculan en dimensiones originales.
    """
    # Redimensionar predicción si es necesario
    if mascara_pred.shape != mascara_gt.shape:
        from skimage.transform import resize

        mascara_pred_resized = resize(
            mascara_pred.astype(float),
            mascara_gt.shape,
            order=0,  # Nearest neighbor preserva naturaleza binaria
            preserve_range=True,
            anti_aliasing=False
        ) > 0.5

        mascara_pred_resized = mascara_pred_resized.astype(bool)
    else:
        mascara_pred_resized = mascara_pred

    # Aplanar para operaciones
    pred_flat = mascara_pred_resized.flatten()
    gt_flat = mascara_gt.flatten()

    # Calcular intersección y unión
    interseccion = np.logical_and(pred_flat, gt_flat).sum()
    union = np.logical_or(pred_flat, gt_flat).sum()
    pred_sum = pred_flat.sum()
    gt_sum = gt_flat.sum()

    # IoU (Intersection over Union)
    iou = interseccion / union if union > 0 else 0.0

    # Dice Coefficient
    dice = (2.0 * interseccion) / (pred_sum + gt_sum) if (pred_sum + gt_sum) > 0 else 0.0

    # Precision (de los píxeles que predije, cuántos son correctos)
    precision = interseccion / pred_sum if pred_sum > 0 else 0.0

    # Recall (de los píxeles GT, cuántos detecté)
    recall = interseccion / gt_sum if gt_sum > 0 else 0.0

    # F1-Score
    f1_score = (2.0 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    return {
        'iou': float(iou),
        'dice': float(dice),
        'precision': float(precision),
        'recall': float(recall),
        'f1_score': float(f1_score)
    }


def calcular_metricas_shapely(geometria: Polygon | MultiPolygon, dimensiones_imagen: Tuple[int, int], logger: Optional[logging.Logger] = None) -> Dict[str, float]:
    """
    Calcula métricas geométricas usando Shapely.

    Métricas de forma (13):
    - área, perímetro, centroide, bounding box
    - aspect_ratio, orientación
    - solidity, compacidad, circularity
    - elongation, rectangularity
    - num_componentes, ratio_componente_principal

    Métricas compositivas (4):
    - distancia_centro, zona_tercios
    - recorte_bordes, espacio_negativo

    Args:
        geometria: Polygon o MultiPolygon de Shapely
        dimensiones_imagen: Tupla (altura, ancho) de la imagen
        logger: Logger opcional

    Returns:
        Diccionario con ~17 métricas geométricas

    Notes:
        - Solidity: compacidad considerando concavidades
        - Compacidad: círculo perfecto = 1.0
        - Zona tercios: 1-9 según regla fotográfica de tercios
    """
    try:
        altura_img, ancho_img = dimensiones_imagen

        # Métricas básicas
        area = geometria.area
        perimetro = geometria.length
        centroide = geometria.centroid
        bbox = geometria.bounds  # (minx, miny, maxx, maxy)

        ancho_bbox = bbox[2] - bbox[0]
        alto_bbox = bbox[3] - bbox[1]
        aspect_ratio = ancho_bbox / alto_bbox if alto_bbox > 0 else 0.0

        # Orientación
        try:
            coords = np.array(geometria.exterior.coords if isinstance(geometria, Polygon) else geometria.geoms[0].exterior.coords)
            orientacion = np.arctan2(coords[-1, 1] - coords[0, 1], coords[-1, 0] - coords[0, 0]) * 180 / np.pi
        except:
            orientacion = 0.0

        # Solidity
        convex_hull = geometria.convex_hull
        solidity = area / convex_hull.area if convex_hull.area > 0 else 0.0

        # Compacidad
        compacidad = (4 * np.pi * area) / (perimetro ** 2) if perimetro > 0 else 0.0

        circularity = compacidad

        # Elongation
        elongation = max(ancho_bbox, alto_bbox) / min(ancho_bbox, alto_bbox) if min(ancho_bbox, alto_bbox) > 0 else 1.0

        # Rectangularity
        area_bbox = ancho_bbox * alto_bbox
        rectangularity = area / area_bbox if area_bbox > 0 else 0.0

        # Fragmentación
        if isinstance(geometria, MultiPolygon):
            num_componentes = len(geometria.geoms)
            areas_componentes = [g.area for g in geometria.geoms]
            area_principal = max(areas_componentes)
            ratio_componente_principal = area_principal / area if area > 0 else 1.0
        else:
            num_componentes = 1
            area_principal = area
            ratio_componente_principal = 1.0

        # Posición compositiva
        centro_imagen = Point(ancho_img / 2, altura_img / 2)
        distancia_centro = centroide.distance(centro_imagen)

        # Zona de tercios
        tercio_ancho = ancho_img / 3
        tercio_alto = altura_img / 3

        col_tercio = int(centroide.x // tercio_ancho)
        fila_tercio = int(centroide.y // tercio_alto)

        col_tercio = min(col_tercio, 2)
        fila_tercio = min(fila_tercio, 2)

        zona_tercios = fila_tercio * 3 + col_tercio + 1

        # Recorte por bordes
        margen = 10
        recorte_izq = max(0, margen - bbox[0])
        recorte_der = max(0, bbox[2] - (ancho_img - margen))
        recorte_sup = max(0, margen - bbox[1])
        recorte_inf = max(0, bbox[3] - (altura_img - margen))

        recorte_total = recorte_izq + recorte_der + recorte_sup + recorte_inf
        perimetro_bbox = 2 * (ancho_bbox + alto_bbox)
        porcentaje_recorte = recorte_total / perimetro_bbox if perimetro_bbox > 0 else 0.0

        # Espacio negativo
        area_imagen_total = altura_img * ancho_img
        espacio_negativo = 1.0 - (area / area_imagen_total)

        return {
            "area_px": float(area),
            "perimetro_px": float(perimetro),
            "centroide_x": float(centroide.x),
            "centroide_y": float(centroide.y),
            "bbox_xmin": float(bbox[0]),
            "bbox_ymin": float(bbox[1]),
            "bbox_xmax": float(bbox[2]),
            "bbox_ymax": float(bbox[3]),
            "aspect_ratio": float(aspect_ratio),
            "orientacion_grados": float(orientacion),
            "solidity": float(solidity),
            "compacidad": float(compacidad),
            "circularity": float(circularity),
            "elongation": float(elongation),
            "rectangularity": float(rectangularity),
            "num_componentes": int(num_componentes),
            "ratio_componente_principal": float(ratio_componente_principal),
            "distancia_centro_px": float(distancia_centro),
            "zona_tercios": int(zona_tercios),
            "recorte_bordes_porcentaje": float(porcentaje_recorte),
            "espacio_negativo": float(espacio_negativo)
        }

    except Exception as e:
        if logger:
            logger.error(f"Error calculando métricas Shapely: {e}")

        return {k: np.nan for k in [
            "area_px", "perimetro_px", "centroide_x", "centroide_y",
            "bbox_xmin", "bbox_ymin", "bbox_xmax", "bbox_ymax",
            "aspect_ratio", "orientacion_grados", "solidity", "compacidad",
            "circularity", "elongation", "rectangularity", "num_componentes",
            "ratio_componente_principal", "distancia_centro_px", "zona_tercios",
            "recorte_bordes_porcentaje", "espacio_negativo"
        ]}

def calcular_metricas_comparativas(
    geom_pred: Polygon | MultiPolygon,
    geom_gt: Polygon | MultiPolygon,
    mascara_pred: np.ndarray,
    mascara_gt: np.ndarray,
    logger: Optional[logging.Logger] = None
) -> Dict[str, float]:
    """
    Calcula métricas de comparación entre predicción y ground truth.

    Combina análisis geométrico (Shapely) y análisis de bordes (arrays).
    Si las máscaras tienen dimensiones diferentes, redimensiona temporalmente
    la predicción para métricas de borde.

    Métricas de contorno (3):
    - boundary_iou: IoU calculado solo sobre bordes
    - hausdorff_distance: Máxima desviación entre contornos
    - chamfer_distance: Desviación promedio

    Métricas de forma (4):
    - desplazamiento_centroide: Distancia Euclidiana entre centroides
    - diferencia_orientacion: Diferencia angular en grados
    - ratio_solidity: pred_solidity / gt_solidity
    - diferencia_compacidad: Diferencia absoluta

    Métricas de área (1):
    - symmetric_difference_area: Área de diferencia simétrica

    Args:
        geom_pred: Geometría predicha (en su resolución original)
        geom_gt: Geometría ground truth
        mascara_pred: Máscara predicha original (H1, W1)
        mascara_gt: Máscara ground truth (H2, W2)
        logger: Logger opcional

    Returns:
        Diccionario con 8 métricas comparativas

    Notes:
        - Métricas geométricas (Shapely) se calculan SIN redimensionar
        - Boundary IoU requiere redimensionamiento temporal para comparación píxel a píxel
        - Hausdorff: Detecta outliers en la segmentación
        - Chamfer: Métrica más robusta a outliers que Hausdorff
    """
    try:
        # Redimensionar predicción si es necesario (SOLO para métricas de borde)
        if mascara_pred.shape != mascara_gt.shape:
            from skimage.transform import resize

            mascara_pred_resized = resize(
                mascara_pred.astype(float),
                mascara_gt.shape,
                order=0,  # Nearest neighbor preserva naturaleza binaria
                preserve_range=True,
                anti_aliasing=False
            ) > 0.5

            mascara_pred_resized = mascara_pred_resized.astype(bool)
        else:
            mascara_pred_resized = mascara_pred

        # --- MÉTRICAS DE BORDE (requieren mismo tamaño) ---

        # Boundary IoU usando Canny
        boundary_pred = cv2.Canny(mascara_pred_resized.astype(np.uint8) * 255, 100, 200) > 0
        boundary_gt = cv2.Canny(mascara_gt.astype(np.uint8) * 255, 100, 200) > 0

        boundary_interseccion = np.logical_and(boundary_pred, boundary_gt).sum()
        boundary_union = np.logical_or(boundary_pred, boundary_gt).sum()

        boundary_iou = boundary_interseccion / boundary_union if boundary_union > 0 else 0.0

        # Hausdorff y Chamfer Distance
        coords_pred = np.column_stack(np.where(boundary_pred))
        coords_gt = np.column_stack(np.where(boundary_gt))

        if len(coords_pred) > 0 and len(coords_gt) > 0:
            hausdorff_dist = max(
                directed_hausdorff(coords_pred, coords_gt)[0],
                directed_hausdorff(coords_gt, coords_pred)[0]
            )

            dist_pred_to_gt = distance_transform_edt(~boundary_gt)[boundary_pred]
            dist_gt_to_pred = distance_transform_edt(~boundary_pred)[boundary_gt]

            chamfer_dist = (dist_pred_to_gt.sum() + dist_gt_to_pred.sum()) / (len(dist_pred_to_gt) + len(dist_gt_to_pred))
        else:
            hausdorff_dist = np.nan
            chamfer_dist = np.nan

        # --- MÉTRICAS GEOMÉTRICAS (Shapely - NO requieren redimensionamiento) ---

        # Desplazamiento centroide
        centroide_pred = geom_pred.centroid
        centroide_gt = geom_gt.centroid
        desplazamiento_centroide = centroide_pred.distance(centroide_gt)

        # Diferencia orientación
        try:
            coords_pred_poly = np.array(geom_pred.exterior.coords if isinstance(geom_pred, Polygon) else geom_pred.geoms[0].exterior.coords)
            coords_gt_poly = np.array(geom_gt.exterior.coords if isinstance(geom_gt, Polygon) else geom_gt.geoms[0].exterior.coords)

            orient_pred = np.arctan2(coords_pred_poly[-1, 1] - coords_pred_poly[0, 1], coords_pred_poly[-1, 0] - coords_pred_poly[0, 0]) * 180 / np.pi
            orient_gt = np.arctan2(coords_gt_poly[-1, 1] - coords_gt_poly[0, 1], coords_gt_poly[-1, 0] - coords_gt_poly[0, 0]) * 180 / np.pi

            diferencia_orientacion = abs(orient_pred - orient_gt)
        except:
            diferencia_orientacion = np.nan

        # Ratio solidity
        solidity_pred = geom_pred.area / geom_pred.convex_hull.area if geom_pred.convex_hull.area > 0 else 0.0
        solidity_gt = geom_gt.area / geom_gt.convex_hull.area if geom_gt.convex_hull.area > 0 else 0.0

        ratio_solidity = solidity_pred / solidity_gt if solidity_gt > 0 else np.nan

        # Diferencia compacidad
        perimetro_pred = geom_pred.length
        perimetro_gt = geom_gt.length

        compacidad_pred = (4 * np.pi * geom_pred.area) / (perimetro_pred ** 2) if perimetro_pred > 0 else 0.0
        compacidad_gt = (4 * np.pi * geom_gt.area) / (perimetro_gt ** 2) if perimetro_gt > 0 else 0.0

        diferencia_compacidad = abs(compacidad_pred - compacidad_gt)

        # Symmetric difference
        sym_diff = geom_pred.symmetric_difference(geom_gt)
        symmetric_difference_area = sym_diff.area

        return {
            "boundary_iou": float(boundary_iou),
            "hausdorff_distance": float(hausdorff_dist),
            "chamfer_distance": float(chamfer_dist),
            "desplazamiento_centroide_px": float(desplazamiento_centroide),
            "diferencia_orientacion_grados": float(diferencia_orientacion),
            "ratio_solidity": float(ratio_solidity),
            "diferencia_compacidad": float(diferencia_compacidad),
            "symmetric_difference_area_px": float(symmetric_difference_area)
        }

    except Exception as e:
        if logger:
            logger.error(f"Error calculando métricas comparativas: {e}")

        return {
            "boundary_iou": np.nan,
            "hausdorff_distance": np.nan,
            "chamfer_distance": np.nan,
            "desplazamiento_centroide_px": np.nan,
            "diferencia_orientacion_grados": np.nan,
            "ratio_solidity": np.nan,
            "diferencia_compacidad": np.nan,
            "symmetric_difference_area_px": np.nan
        }

def calcular_metricas_mahotas(imagen_rgb: np.ndarray, mascara: np.ndarray, ancho_borde: int = 5, max_dimension: int = 1024, logger: Optional[logging.Logger] = None) -> Dict[str, float]:
    """
    Calcula métricas texturales usando Mahotas.

    Extrae 3 tipos de métricas:
    1. Texturas Haralick interior (13 features)
    2. Texturas Haralick borde (13 features)
    3. Estadísticas básicas (4 valores)

    Args:
        imagen_rgb: Imagen original en RGB
        mascara: Máscara binaria de la región
        ancho_borde: Ancho del borde en pixels
        max_dimension: Redimensión máxima (optimización)
        logger: Logger opcional

    Returns:
        Diccionario con 30 métricas texturales

    Notes:
        - Haralick: Calcula co-ocurrencia de grises
        - Borde: Evalúa calidad de transición persona-fondo
        - Redimensión: Trade-off velocidad vs precisión

    References:
        Haralick et al. (1973). Textural Features for Image Classification
    """
    try:
        altura, ancho = imagen_rgb.shape[:2]
        factor_escala = 1.0

        # Redimensionar si excede límite
        if max(altura, ancho) > max_dimension:
            factor_escala = max_dimension / max(altura, ancho)
            nuevo_ancho = int(ancho * factor_escala)
            nuevo_alto = int(altura * factor_escala)

            imagen_rgb = cv2.resize(imagen_rgb, (nuevo_ancho, nuevo_alto), interpolation=cv2.INTER_AREA)
            mascara = cv2.resize(mascara.astype(np.uint8), (nuevo_ancho, nuevo_alto), interpolation=cv2.INTER_NEAREST).astype(bool)

        # Convertir a escala de grises
        imagen_gris = cv2.cvtColor(imagen_rgb, cv2.COLOR_RGB2GRAY)

        # Región interior
        region_persona = imagen_gris * mascara

        if region_persona[mascara].size == 0:
            raise ValueError("Región de persona vacía")

        haralick_interior = mahotas.features.haralick(region_persona, ignore_zeros=True).mean(axis=0)

        # Región borde
        kernel = np.ones((ancho_borde * 2 + 1, ancho_borde * 2 + 1), np.uint8)
        mascara_dilatada = cv2.dilate(mascara.astype(np.uint8), kernel, iterations=1).astype(bool)
        mascara_erosionada = cv2.erode(mascara.astype(np.uint8), kernel, iterations=1).astype(bool)

        mascara_borde = np.logical_and(mascara_dilatada, ~mascara_erosionada)

        region_borde = imagen_gris * mascara_borde

        if region_borde[mascara_borde].size == 0:
            haralick_borde = np.full(13, np.nan)
        else:
            haralick_borde = mahotas.features.haralick(region_borde, ignore_zeros=True).mean(axis=0)

        # Estadísticas básicas
        intensidades = region_persona[mascara]

        media_intensidad = float(np.mean(intensidades))
        std_intensidad = float(np.std(intensidades))
        min_intensidad = float(np.min(intensidades))
        max_intensidad = float(np.max(intensidades))

        # Construir diccionario de resultados
        nombres_haralick = [
            "angular_second_moment", "contrast", "correlation",
            "sum_of_squares_variance", "inverse_difference_moment",
            "sum_average", "sum_variance", "sum_entropy",
            "entropy", "difference_variance", "difference_entropy",
            "information_measure_correlation_1", "information_measure_correlation_2"
        ]

        resultado = {}

        for i, nombre in enumerate(nombres_haralick):
            resultado[f"haralick_interior_{nombre}"] = float(haralick_interior[i])
            resultado[f"haralick_borde_{nombre}"] = float(haralick_borde[i])

        resultado.update({
            "intensidad_media": media_intensidad,
            "intensidad_std": std_intensidad,
            "intensidad_min": min_intensidad,
            "intensidad_max": max_intensidad
        })

        return resultado

    except Exception as e:
        if logger:
            logger.error(f"Error calculando métricas Mahotas: {e}")

        nombres_haralick = [
            "angular_second_moment", "contrast", "correlation",
            "sum_of_squares_variance", "inverse_difference_moment",
            "sum_average", "sum_variance", "sum_entropy",
            "entropy", "difference_variance", "difference_entropy",
            "information_measure_correlation_1", "information_measure_correlation_2"
        ]

        resultado = {}
        for nombre in nombres_haralick:
            resultado[f"haralick_interior_{nombre}"] = np.nan
            resultado[f"haralick_borde_{nombre}"] = np.nan

        resultado.update({
            "intensidad_media": np.nan,
            "intensidad_std": np.nan,
            "intensidad_min": np.nan,
            "intensidad_max": np.nan
        })

        return resultado

In [ ]:
# =============================================================================
# SISTEMA DE SELECCIÓN INTERACTIVA
# =============================================================================

class SelectorModelos:
    """
    Sistema de selección interactiva de familias de modelos a evaluar.
    """

    def __init__(self, indice_maestro: Dict, logger: logging.Logger):
        """
        Inicializa el selector.

        Args:
            indice_maestro: Índice maestro consolidado de Fase 1
            logger: Sistema de logging
        """
        self.indice = indice_maestro
        self.logger = logger
        self.modelos_por_familia = self._extraer_familias()

    def _extraer_familias(self) -> Dict[str, List[str]]:
        """
        Extrae configuraciones agrupadas por familia desde índice maestro.

        Returns:
            Diccionario {familia: [lista_de_codigos_config]}
        """
        familias = {}

        # Iterar sobre todas las fotos
        for foto_id, datos_foto in self.indice.items():
            modelos_disponibles = datos_foto.get('modelos_disponibles', {})

            # Iterar sobre todas las configuraciones de esta foto
            for codigo_config in modelos_disponibles.keys():
                # Extraer familia del código
                familia = self._extraer_familia_de_codigo(codigo_config)

                if familia not in familias:
                    familias[familia] = set()

                familias[familia].add(codigo_config)

        # Convertir sets a listas ordenadas
        familias_listas = {
            familia: sorted(list(configs))
            for familia, configs in familias.items()
        }

        return familias_listas

    @staticmethod
    def _extraer_familia_de_codigo(codigo_config: str) -> str:
        """
        Extrae el nombre de la familia del modelo desde el código de configuración.

        Ejemplos reales de tu índice:
        - "mask2former_large_ade_baja_sensibilidad" -> "mask2former"
        - "bodypix_mobilenet_050_estandar" -> "bodypix"
        - "sam2_hiera_large_auto_generacion" -> "sam2"
        - "sam2_hiera_large_box_prompts" -> "sam2_prompts"
        - "yolov8_nano_estandar" -> "yolov8"
        - "oneformer_dinat_large_semantic" -> "oneformer"

        Args:
            codigo_config: Código de configuración completo

        Returns:
            Nombre de la familia del modelo
        """
        # Casos especiales: SAM2 con prompts
        if codigo_config.startswith('sam2'):
            # Si contiene palabras clave de prompts, es sam2_prompts
            if any(palabra in codigo_config for palabra in ['box', 'point', 'combined', 'prompts']):
                return 'sam2_prompts'
            else:
                # Es SAM2 auto (sin prompts)
                return 'sam2'

        # Caso general: primera palabra antes del primer guión bajo
        return codigo_config.split('_')[0]

    def mostrar_menu(self) -> int:
        """
        Muestra menú interactivo de selección.

        Returns:
            Índice de opción seleccionada, o -1 si cancela
        """
        print("\n" + "="*80)
        print("SELECCIÓN DE MODELOS PARA PROCESAMIENTO")
        print("="*80)

        if not self.modelos_por_familia:
            print("\n¡ERROR! No se encontraron configuraciones en el índice maestro")
            return -1

        # Orden preferido de familias
        familias_ordenadas = ['bodypix', 'mask2former', 'oneformer', 'sam2', 'sam2_prompts', 'yolov8']
        familias_disponibles = [f for f in familias_ordenadas if f in self.modelos_por_familia]

        # Agregar familias que no estén en el orden preferido (por si acaso)
        for familia in sorted(self.modelos_por_familia.keys()):
            if familia not in familias_disponibles:
                familias_disponibles.append(familia)

        print("\nModelos disponibles:\n")

        for idx, familia in enumerate(familias_disponibles, start=1):
            num_configs = len(self.modelos_por_familia[familia])
            nombre_display = familia.upper().replace('_', ' ')
            print(f"{idx}. {nombre_display}")
            print(f"   Configuraciones: {num_configs}\n")

        print(f"{len(familias_disponibles) + 1}. TODOS LOS MODELOS")
        total_configs = sum(len(configs) for configs in self.modelos_por_familia.values())
        print(f"   Total: {total_configs} configuraciones\n")

        print(f"{len(familias_disponibles) + 2}. SELECCIÓN MÚLTIPLE (separar con comas)")

        print("\n" + "="*80)

        while True:
            try:
                seleccion = input(f"\nSeleccione opción [1-{len(familias_disponibles) + 2}]: ").strip()

                if not seleccion:
                    print("Selección vacía. Cancelando...")
                    return -1

                opcion = int(seleccion)

                if 1 <= opcion <= len(familias_disponibles) + 2:
                    return opcion
                else:
                    print(f"Por favor ingrese un número entre 1 y {len(familias_disponibles) + 2}")

            except ValueError:
                print("Entrada inválida. Por favor ingrese un número")

            except KeyboardInterrupt:
                print("\n\nSelección cancelada por el usuario")
                return -1

    def seleccion_multiple(self) -> List[str]:
        """
        Permite selección múltiple de familias.

        Returns:
            Lista de familias seleccionadas
        """
        familias_ordenadas = ['bodypix', 'mask2former', 'oneformer', 'sam2', 'sam2_prompts', 'yolov8']
        familias_disponibles = [f for f in familias_ordenadas if f in self.modelos_por_familia]

        # Agregar familias adicionales no ordenadas
        for familia in sorted(self.modelos_por_familia.keys()):
            if familia not in familias_disponibles:
                familias_disponibles.append(familia)

        print("\n" + "="*80)
        print("SELECCIÓN MÚLTIPLE")
        print("="*80)
        print("\nIngrese los números separados por comas (ej: 1,3,5)\n")

        for idx, familia in enumerate(familias_disponibles, start=1):
            nombre_display = familia.upper().replace('_', ' ')
            num_configs = len(self.modelos_por_familia[familia])
            print(f"{idx}. {nombre_display} ({num_configs} configs)")

        seleccion = input("\nSelección: ").strip()

        if not seleccion:
            return []

        try:
            indices = [int(x.strip()) for x in seleccion.split(',')]
            familias_sel = []

            for idx in indices:
                if 1 <= idx <= len(familias_disponibles):
                    familias_sel.append(familias_disponibles[idx - 1])
                else:
                    print(f"Advertencia: índice {idx} fuera de rango, ignorado")

            return familias_sel

        except ValueError:
            print("Error: formato inválido")
            return []

    def expandir_seleccion(self, familias_seleccionadas: List[str]) -> List[str]:
        """
        Expande familias seleccionadas a lista completa de configuraciones.

        Args:
            familias_seleccionadas: Lista de códigos de familia

        Returns:
            Lista completa de códigos de configuración
        """
        if 'TODAS' in familias_seleccionadas:
            configs = []
            for familia_configs in self.modelos_por_familia.values():
                configs.extend(familia_configs)
            return sorted(configs)

        configs = []
        for familia in familias_seleccionadas:
            if familia in self.modelos_por_familia:
                configs.extend(self.modelos_por_familia[familia])

        return sorted(configs)

    def confirmar_seleccion(self, configuraciones: List[str]) -> bool:
        """
        Muestra resumen y solicita confirmación.

        Args:
            configuraciones: Lista de configuraciones seleccionadas

        Returns:
            True si usuario confirma, False si cancela
        """
        print("\n" + "="*80)
        print("RESUMEN DE SELECCIÓN")
        print("="*80)
        print(f"\nTotal de configuraciones seleccionadas: {len(configuraciones)}")

        if len(configuraciones) <= 20:
            print("\nConfiguraciones:")
            for config in configuraciones:
                print(f"  - {config}")
        else:
            print("\nPrimeras 10 configuraciones:")
            for config in configuraciones[:10]:
                print(f"  - {config}")
            print(f"  ... y {len(configuraciones) - 10} más")

        print("\n" + "="*80)

        while True:
            confirmacion = input("\n¿Continuar con esta selección? [S/n]: ").strip().lower()

            if confirmacion in ['s', 'si', 'yes', '']:
                return True
            elif confirmacion in ['n', 'no']:
                return False
            else:
                print("Por favor responda 's' o 'n'")

    def seleccionar(self) -> Optional[List[str]]:
        """
        Proceso completo de selección interactiva.

        Returns:
            Lista de configuraciones seleccionadas, o None si se cancela
        """
        opcion = self.mostrar_menu()

        if opcion == -1:
            return None

        familias_ordenadas = ['bodypix', 'mask2former', 'oneformer', 'sam2', 'sam2_prompts', 'yolov8']
        familias_disponibles = [f for f in familias_ordenadas if f in self.modelos_por_familia]

        # Agregar familias adicionales
        for familia in sorted(self.modelos_por_familia.keys()):
            if familia not in familias_disponibles:
                familias_disponibles.append(familia)

        if opcion == len(familias_disponibles) + 1:
            # TODAS
            familias_sel = ['TODAS']
        elif opcion == len(familias_disponibles) + 2:
            # SELECCIÓN MÚLTIPLE
            familias_sel = self.seleccion_multiple()
            if not familias_sel:
                print("\nSelección cancelada")
                return None
        else:
            # FAMILIA INDIVIDUAL
            familias_sel = [familias_disponibles[opcion - 1]]

        configuraciones = self.expandir_seleccion(familias_sel)

        if not configuraciones:
            print("\nError: No se seleccionaron configuraciones válidas")
            return None

        if self.confirmar_seleccion(configuraciones):
            return configuraciones
        else:
            print("\nSelección cancelada")
            return None

In [ ]:
# =============================================================================
# FUNCIÓN AUXILIAR PARA EXTRACCIÓN DE MODELO
# =============================================================================

def extraer_modelo_desde_config(config_codigo: str) -> str:
    """
    Extrae el nombre del modelo desde el código de configuración.

    Implementa lógica especial para SAM2 que distingue entre modo automático
    y modo con prompts, ya que tienen comportamientos y rendimientos muy diferentes.

    Casos especiales:
    - sam2_prompts_* → 'sam2_prompts' (modo con guía de dominio)
    - sam2_* (sin prompts) → 'sam2_auto' (modo automático)
    - Otros modelos → primer segmento del código

    Args:
        config_codigo: Código de configuración completo
            Ejemplos:
            - 'sam2_prompts_large_saliency_conservative'
            - 'sam2_base_plus_balanced'
            - 'yolov8_xlarge_balanced_portrait'
            - 'oneformer_coco_swin_semantic_t040'

    Returns:
        Nombre del modelo normalizado para agrupación en análisis

    Examples:
        >>> extraer_modelo_desde_config('sam2_prompts_large_saliency')
        'sam2_prompts'
        >>> extraer_modelo_desde_config('sam2_base_plus_balanced')
        'sam2_auto'
        >>> extraer_modelo_desde_config('yolov8_xlarge_balanced')
        'yolov8'
    """
    if config_codigo.startswith('sam2_prompts_'):
        return 'sam2_prompts'
    elif config_codigo.startswith('sam2_'):
        return 'sam2_auto'
    else:
        return config_codigo.split('_')[0]

In [ ]:
# =============================================================================
# CLASE: ORQUESTADOR PRINCIPAL DE FASE 2A (CORREGIDA)
# =============================================================================

class OrquestadorFase2A:
    """
    Orquestador principal del pipeline de evaluación Fase 2A.

    Coordina el pipeline completo de evaluación cuantitativa:
    - Gestión de checkpoint para procesamiento resumible
    - Carga de ground truth y máscaras predichas
    - Cálculo de métricas clásicas, geométricas y texturales
    - Consolidación de resultados en múltiples formatos
    - Generación de estadísticas y rankings

    Attributes:
        config: Configuración global de Fase 2A
        configuraciones_seleccionadas: Lista de códigos de configuración a procesar
        logger: Logger configurado según nivel de verbosidad
        checkpoint: Sistema de checkpoint para reanudar procesamiento
        resultados_consolidados: Lista acumulativa de resultados para CSV
        estadisticas_omisiones: Registro de configuraciones omitidas por foto
        indice_maestro: Índice maestro cargado desde Fase 1
        fotografias_disponibles: Lista de estructuras con información de fotografías

    Notes:
        - Las configuraciones con ruta_mascara=null se omiten automáticamente
        - El checkpoint se actualiza después de cada fotografía procesada
        - Los errores en cálculo de métricas se registran como NaN sin detener ejecución
        - SAM2 Auto y SAM2 Prompts se distinguen como modelos separados

    Version:
        2.0 - Corregida distinción SAM2 auto vs SAM2 prompts
    """

    def __init__(self, config: ConfiguracionFase2A, configuraciones: List[str]):
        """
        Inicializa el orquestador con configuración y lista de configuraciones.

        Args:
            config: Configuración global del sistema
            configuraciones: Lista de códigos de configuración seleccionados por el usuario

        Raises:
            FileNotFoundError: Si no existe el índice maestro de Fase 1
        """
        self.config = config
        self.configuraciones_seleccionadas = configuraciones
        self.logger = configurar_logging(config.nivel_logging, "OrquestadorFase2A")

        config.crear_directorios()

        self.checkpoint: Optional[CheckpointFase2A] = None
        self.resultados_consolidados: List[Dict] = []
        self.estadisticas_omisiones: Dict[str, List[str]] = {}

        # Cargar índice maestro de Fase 1
        ruta_indice = config.ruta_fase1 / "indice_maestro.json"

        if not ruta_indice.exists():
            raise FileNotFoundError(f"Índice maestro no encontrado: {ruta_indice}")

        with open(ruta_indice, 'r', encoding='utf-8') as f:
            self.indice_maestro = json.load(f)

        # Extraer información de fotografías disponibles
        self.fotografias_disponibles = []

        for foto_id in sorted(self.indice_maestro.keys()):
            datos_foto = self.indice_maestro[foto_id]

            foto_info = {
                "codigo_foto": foto_id,
                "ruta_imagen": datos_foto["rutas"]["imagen"],
                "ruta_ground_truth": datos_foto["rutas"]["ground_truth"],
                "ruta_caracteristicas": datos_foto["rutas"]["caracteristicas"],
                "modelos_disponibles": datos_foto["modelos_disponibles"]
            }

            self.fotografias_disponibles.append(foto_info)

        self.logger.info(f"Índice maestro cargado: {len(self.fotografias_disponibles)} fotografías")

    def cargar_checkpoint(self) -> None:
        """
        Carga checkpoint existente si está habilitado.

        Permite reanudar procesamiento desde fotografía interrumpida.
        Actualiza la lista de fotografías pendientes basándose en fotos ya procesadas.
        """
        ruta_checkpoint = self.config.ruta_salida / "checkpoint_fase2a.json"

        if self.config.usar_checkpoint and ruta_checkpoint.exists():
            try:
                with open(ruta_checkpoint, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    self.checkpoint = CheckpointFase2A.from_dict(data)

                self.logger.info(f"Checkpoint cargado: {len(self.checkpoint.fotos_procesadas)} fotos ya procesadas")

            except Exception as e:
                self.logger.warning(f"Error cargando checkpoint: {e}. Iniciando desde cero")
                self.checkpoint = CheckpointFase2A()
        else:
            self.checkpoint = CheckpointFase2A()

        # Actualizar fotos pendientes
        codigos_fotos = [f["codigo_foto"] for f in self.fotografias_disponibles]

        self.checkpoint.fotos_pendientes = [
            codigo for codigo in codigos_fotos
            if codigo not in self.checkpoint.fotos_procesadas
        ]

        self.checkpoint.configuraciones_totales = len(self.configuraciones_seleccionadas)

    def guardar_checkpoint(self) -> None:
        """
        Guarda estado actual del checkpoint.

        Se ejecuta después de cada fotografía procesada para permitir
        reanudación en caso de interrupción.
        """
        if not self.config.usar_checkpoint:
            return

        self.checkpoint.ultima_actualizacion = datetime.now().isoformat()

        ruta_checkpoint = self.config.ruta_salida / "checkpoint_fase2a.json"

        guardar_json_seguro(self.checkpoint.to_dict(), ruta_checkpoint, self.logger)

    def procesar_fotografia(self, info_foto: Dict) -> None:
        """
        Procesa una fotografía completa con todas sus configuraciones seleccionadas.

        Pipeline de procesamiento:
        1. Carga ground truth
        2. Convierte GT a geometría Shapely
        3. Calcula métricas base del GT
        4. Para cada configuración seleccionada disponible:
           - Verifica existencia de máscara NPZ
           - Carga máscara predicha (con selección automática para SAM2)
           - Calcula métricas clásicas (IoU, Dice, Precision, Recall, F1)
           - Calcula métricas geométricas Shapely
           - Calcula métricas comparativas
           - Calcula métricas texturales Mahotas
        5. Guarda resultados individuales en JSON
        6. Actualiza checkpoint

        Args:
            info_foto: Diccionario con información de la fotografía:
                - codigo_foto: Identificador único
                - ruta_imagen: Path a imagen original
                - ruta_ground_truth: Path a máscara GT
                - ruta_caracteristicas: Path a JSON de características
                - modelos_disponibles: Dict de configuraciones disponibles

        Notes:
            - Configuraciones sin máscara NPZ (ruta_mascara=null) se omiten
            - Errores en conversión a Shapely se registran como omisiones
            - Mahotas se omite si no existe imagen original RGB
            - SAM2 Auto y SAM2 Prompts se distinguen como modelos separados
        """
        codigo_foto = info_foto["codigo_foto"]

        if self.config.nivel_logging in [NivelLogging.NORMAL, NivelLogging.DETAILED, NivelLogging.DEBUG]:
            self.logger.info(f"\n{'='*80}")
            self.logger.info(f"Procesando fotografía: {codigo_foto}")
            self.logger.info(f"{'='*80}")

        # Cargar ground truth
        ruta_gt = Path(info_foto["ruta_ground_truth"])

        if not ruta_gt.exists():
            self.logger.error(f"Ground truth no encontrado para {codigo_foto}. Omitiendo fotografía")
            return

        # MODIFICADO: Ahora retorna tuple (mascara, metadata)
        mascara_gt, _ = cargar_mascara_npz(ruta_gt, logger=self.logger)

        if mascara_gt is None:
            self.logger.error(f"Error cargando ground truth de {codigo_foto}. Omitiendo fotografía")
            return

        # Convertir GT a Shapely
        geom_gt = mascara_a_shapely(mascara_gt, self.logger)

        if geom_gt is None:
            self.logger.error(f"Error convirtiendo GT a Shapely para {codigo_foto}. Omitiendo fotografía")
            return

        # Dimensiones de la imagen
        dimensiones_imagen = (mascara_gt.shape[0], mascara_gt.shape[1])

        # Métricas del ground truth
        metricas_gt = calcular_metricas_shapely(geom_gt, dimensiones_imagen, self.logger)

        # Cargar imagen original para Mahotas
        ruta_imagen_original = Path(info_foto["ruta_imagen"])

        if ruta_imagen_original.exists():
            imagen_rgb = np.array(Image.open(ruta_imagen_original).convert("RGB"))
        else:
            self.logger.warning(f"Imagen original no encontrada: {ruta_imagen_original}. Mahotas omitido")
            imagen_rgb = None

        # Estructura de resultados
        resultados_foto = {
            "codigo_foto": codigo_foto,
            "dimensiones_imagen": {
                "altura": dimensiones_imagen[0],
                "ancho": dimensiones_imagen[1]
            },
            "ground_truth_metricas": metricas_gt,
            "configuraciones": []
        }

        configs_procesadas = 0
        configs_omitidas = 0

        # Iterar sobre configuraciones seleccionadas que existen en esta foto
        modelos_disponibles_foto = info_foto["modelos_disponibles"]

        for config_codigo in self.configuraciones_seleccionadas:
            # Verificar si esta configuración existe para esta foto
            if config_codigo not in modelos_disponibles_foto:
                if self.config.nivel_logging == NivelLogging.DEBUG:
                    self.logger.debug(f"  Config {config_codigo} no disponible para {codigo_foto}")
                configs_omitidas += 1

                # Registrar omisión - CORREGIDO: usar función auxiliar
                modelo = extraer_modelo_desde_config(config_codigo)
                clave_omision = f"{codigo_foto}_{config_codigo}"
                if modelo not in self.estadisticas_omisiones:
                    self.estadisticas_omisiones[modelo] = []
                self.estadisticas_omisiones[modelo].append(clave_omision)
                continue

            if self.config.nivel_logging == NivelLogging.DEBUG:
                self.logger.debug(f"  Procesando config: {config_codigo}")

            # Obtener información de la configuración
            config_info = modelos_disponibles_foto[config_codigo]

            # Verificar si tiene máscara NPZ (puede ser null)
            if config_info.get("ruta_mascara") is None:
                if self.config.nivel_logging == NivelLogging.DEBUG:
                    self.logger.debug(f"    Config {config_codigo} no tiene máscara NPZ (solo metadata)")
                configs_omitidas += 1

                # Registrar omisión - CORREGIDO: usar función auxiliar
                modelo = extraer_modelo_desde_config(config_codigo)
                clave_omision = f"{codigo_foto}_{config_codigo}"
                if modelo not in self.estadisticas_omisiones:
                    self.estadisticas_omisiones[modelo] = []
                self.estadisticas_omisiones[modelo].append(clave_omision)
                continue

            ruta_mascara = Path(config_info["ruta_mascara"])

            if not ruta_mascara.exists():
                self.logger.warning(f"    Máscara no encontrada: {ruta_mascara}")
                configs_omitidas += 1
                continue

            # MODIFICADO: Ahora retorna tuple (mascara, metadata_seleccion)
            mascara_pred, metadata_seleccion = cargar_mascara_npz(ruta_mascara, logger=self.logger)

            if mascara_pred is None:
                self.logger.warning(f"    Error cargando máscara {config_codigo} para {codigo_foto}")
                configs_omitidas += 1
                continue

            # Calcular métricas clásicas
            metricas_clasicas = calcular_metricas_clasicas(mascara_pred, mascara_gt)

            # Convertir predicción a Shapely
            geom_pred = mascara_a_shapely(mascara_pred, self.logger)

            if geom_pred is None:
                self.logger.warning(f"    Error convirtiendo predicción a Shapely: {config_codigo}")
                configs_omitidas += 1
                continue

            # Métricas Shapely de predicción
            metricas_shapely_pred = calcular_metricas_shapely(geom_pred, dimensiones_imagen, self.logger)

            # Métricas comparativas
            metricas_comparativas = calcular_metricas_comparativas(
                geom_pred, geom_gt, mascara_pred, mascara_gt, self.logger
            )

            # Métricas Mahotas
            if imagen_rgb is not None:
                metricas_mahotas = calcular_metricas_mahotas(
                    imagen_rgb, mascara_pred,
                    ancho_borde=self.config.ancho_borde_px,
                    max_dimension=self.config.max_dimension_mahotas,
                    logger=self.logger
                )
            else:
                metricas_mahotas = {}

            # CORREGIDO: Consolidar resultados con modelo extraído correctamente
            resultado_config = {
                "config_codigo": config_codigo,
                "modelo": extraer_modelo_desde_config(config_codigo),  # CORREGIDO
                "metadata_seleccion": metadata_seleccion,
                "metricas_clasicas": metricas_clasicas,
                "metricas_shapely": metricas_shapely_pred,
                "metricas_comparativas": metricas_comparativas,
                "metricas_mahotas": metricas_mahotas
            }

            resultados_foto["configuraciones"].append(resultado_config)

            # Agregar a consolidado para CSV - CORREGIDO
            fila_csv = {
                "codigo_foto": codigo_foto,
                "config_codigo": config_codigo,
                "modelo": extraer_modelo_desde_config(config_codigo),  # CORREGIDO
                **metricas_clasicas,
                **metricas_shapely_pred,
                **metricas_comparativas,
                **metricas_mahotas
            }

            # NUEVO: Añadir campos de metadata de selección al CSV si existen
            if metadata_seleccion is not None:
                fila_csv["seleccion_tipo"] = metadata_seleccion.get("tipo_seleccion", None)
                fila_csv["seleccion_num_candidatos"] = metadata_seleccion.get("num_candidatos", None)
                fila_csv["seleccion_confidence"] = metadata_seleccion.get("confidence_seleccionada", None)
                fila_csv["seleccion_area"] = metadata_seleccion.get("area_seleccionada", None)

                # Para SAM2 Prompts, añadir índice de persona seleccionada
                if "persona_seleccionada" in metadata_seleccion:
                    fila_csv["seleccion_persona_idx"] = metadata_seleccion.get("persona_seleccionada", None)

            self.resultados_consolidados.append(fila_csv)

            configs_procesadas += 1

        # Guardar JSON individual
        if self.config.guardar_json_individuales:
            ruta_json_individual = self.config.ruta_salida / "metricas_individuales" / f"{codigo_foto}_metricas_completas.json"
            guardar_json_seguro(resultados_foto, ruta_json_individual, self.logger)

        # Actualizar checkpoint
        self.checkpoint.fotos_procesadas.add(codigo_foto)

        if codigo_foto in self.checkpoint.fotos_pendientes:
            self.checkpoint.fotos_pendientes.remove(codigo_foto)

        self.guardar_checkpoint()

        if self.config.nivel_logging in [NivelLogging.NORMAL, NivelLogging.DETAILED]:
            self.logger.info(f"  Configuraciones procesadas: {configs_procesadas}")
            self.logger.info(f"  Configuraciones omitidas: {configs_omitidas}")

    def ejecutar(self) -> None:
        """
        Ejecuta el pipeline completo de Fase 2A.

        Procesa todas las fotografías pendientes y genera archivos de salida:
        - JSONs individuales por fotografía (opcional)
        - CSV consolidado con todas las métricas
        - Estadísticas por modelo
        - Ranking TOP-10 de configuraciones
        - Reporte de configuraciones omitidas

        El procesamiento es resumible mediante checkpoint si está habilitado.
        """
        self.logger.info("\nIniciando Fase 2A - Evaluación Cuantitativa")
        self.logger.info(f"Configuraciones a evaluar: {len(self.configuraciones_seleccionadas)}")
        self.logger.info(f"Fotografías disponibles: {len(self.fotografias_disponibles)}")

        self.cargar_checkpoint()

        self.logger.info(f"Fotografías pendientes: {len(self.checkpoint.fotos_pendientes)}")
        self.logger.info(f"Fotografías ya procesadas: {len(self.checkpoint.fotos_procesadas)}\n")

        # Procesar cada fotografía
        for info_foto in self.fotografias_disponibles:
            codigo_foto = info_foto["codigo_foto"]

            if codigo_foto in self.checkpoint.fotos_procesadas:
                if self.config.nivel_logging == NivelLogging.DEBUG:
                    self.logger.debug(f"Omitiendo {codigo_foto} (ya procesada)")
                continue

            try:
                self.procesar_fotografia(info_foto)

            except Exception as e:
                self.logger.error(f"Error procesando {codigo_foto}: {e}")
                import traceback
                traceback.print_exc()
                continue

        self.logger.info("\nProcesamiento de fotografías completado")

        # Generar archivos consolidados
        if self.config.guardar_csv_consolidado:
            self.generar_csv_consolidado()

        if self.config.guardar_estadisticas:
            self.generar_estadisticas_por_modelo()

        self.generar_ranking_top10()

        self.guardar_reporte_omisiones()

        self.logger.info("\nFase 2A completada exitosamente")

    def generar_csv_consolidado(self) -> None:
        """
        Genera CSV consolidado con todas las métricas.

        Formato: Una fila por combinación foto-configuración.
        Columnas: código_foto, config_codigo, modelo + ~60 métricas + metadata selección

        Nota: El campo 'modelo' ahora distingue entre 'sam2_auto' y 'sam2_prompts'

        Output: metricas_agregadas/todas_metricas.csv
        """
        try:
            if len(self.resultados_consolidados) == 0:
                self.logger.warning("No hay datos para consolidar en CSV")
                return

            df = pd.DataFrame(self.resultados_consolidados)

            ruta_csv = self.config.ruta_salida / "metricas_agregadas" / "todas_metricas.csv"

            df.to_csv(ruta_csv, index=False, encoding='utf-8')

            self.logger.info(f"CSV consolidado guardado: {ruta_csv}")
            self.logger.info(f"  Filas: {len(df)}")
            self.logger.info(f"  Columnas: {len(df.columns)}")

            # NUEVO: Mostrar distribución por modelo
            self.logger.info(f"  Distribución por modelo:")
            for modelo, count in df['modelo'].value_counts().items():
                self.logger.info(f"    {modelo}: {count}")

        except Exception as e:
            self.logger.error(f"Error generando CSV consolidado: {e}")

    def generar_ranking_top10(self) -> None:
        """
        Genera ranking TOP-10 de configuraciones ordenadas por IoU medio.

        Para cada configuración calcula:
        - IoU medio, desviación estándar, mínimo, máximo
        - Número de fotografías evaluadas
        - Modelo al que pertenece

        Output: metricas_agregadas/ranking_configuraciones.csv
        """
        try:
            if len(self.resultados_consolidados) == 0:
                self.logger.warning("No hay datos para generar ranking")
                return

            df = pd.DataFrame(self.resultados_consolidados)

            # Agrupar por configuración y calcular estadísticas de IoU
            ranking = df.groupby('config_codigo').agg({
                'iou': ['mean', 'std', 'min', 'max', 'count'],
                'modelo': 'first'
            }).reset_index()

            ranking.columns = ['config_codigo', 'iou_mean', 'iou_std', 'iou_min', 'iou_max', 'num_fotos', 'modelo']

            # Ordenar por IoU medio descendente
            ranking = ranking.sort_values('iou_mean', ascending=False)

            # TOP-10
            top10 = ranking.head(10)

            ruta_ranking = self.config.ruta_salida / "metricas_agregadas" / "ranking_configuraciones.csv"

            top10.to_csv(ruta_ranking, index=False, encoding='utf-8')

            self.logger.info(f"Ranking TOP-10 guardado: {ruta_ranking}")

        except Exception as e:
            self.logger.error(f"Error generando ranking: {e}")

    def generar_estadisticas_por_modelo(self) -> None:
        """
        Genera estadísticas agregadas por modelo.

        Para cada modelo calcula:
        - Número de configuraciones únicas
        - Número total de evaluaciones
        - Estadísticas de IoU: media, desviación, min, max, mediana

        Nota: SAM2 Auto y SAM2 Prompts aparecen como modelos separados

        Output: metricas_agregadas/estadisticas_por_modelo.json
        """
        try:
            if len(self.resultados_consolidados) == 0:
                self.logger.warning("No hay datos suficientes para generar estadísticas")
                return

            df = pd.DataFrame(self.resultados_consolidados)

            ruta_stats = self.config.ruta_salida / "metricas_agregadas" / "estadisticas_por_modelo.json"

            stats_por_modelo = {}

            for modelo in df['modelo'].unique():
                df_modelo = df[df['modelo'] == modelo]

                stats_por_modelo[modelo] = {
                    'num_configuraciones': int(df_modelo['config_codigo'].nunique()),
                    'num_evaluaciones': int(len(df_modelo)),
                    'iou': {
                        'mean': float(df_modelo['iou'].mean()),
                        'std': float(df_modelo['iou'].std()),
                        'min': float(df_modelo['iou'].min()),
                        'max': float(df_modelo['iou'].max()),
                        'median': float(df_modelo['iou'].median())
                    }
                }

            guardar_json_seguro(stats_por_modelo, ruta_stats, self.logger)

            self.logger.info(f"Estadísticas por modelo guardadas: {ruta_stats}")

        except Exception as e:
            self.logger.error(f"Error generando estadísticas: {e}")

    def guardar_reporte_omisiones(self) -> None:
        """
        Guarda reporte detallado de configuraciones omitidas.

        Útil para debugging y auditoría del proceso de evaluación.
        Incluye:
        - Total de omisiones
        - Omisiones por modelo
        - Lista detallada de combinaciones foto-config omitidas

        Output: metricas_agregadas/configuraciones_omitidas.json
        """
        try:
            if len(self.estadisticas_omisiones) == 0:
                self.logger.info("No hay omisiones para reportar")
                return

            ruta_omisiones = self.config.ruta_salida / "metricas_agregadas" / "configuraciones_omitidas.json"

            reporte = {
                "total_omisiones": sum(len(v) for v in self.estadisticas_omisiones.values()),
                "por_modelo": {
                    modelo: {
                        "count": len(omisiones),
                        "combinaciones": omisiones
                    }
                    for modelo, omisiones in self.estadisticas_omisiones.items()
                }
            }

            guardar_json_seguro(reporte, ruta_omisiones, self.logger)

            self.logger.info(f"Reporte de omisiones guardado: {ruta_omisiones}")

        except Exception as e:
            self.logger.error(f"Error guardando reporte de omisiones: {e}")

In [ ]:
# =============================================================================
# FUNCIÓN PRINCIPAL DE EJECUCIÓN INTERACTIVA
# =============================================================================

def ejecutar_fase2a_interactivo(ruta_base_tfm: str = "/content/drive/MyDrive/TFM") -> Optional[OrquestadorFase2A]:
    """
    Ejecuta el pipeline completo de Fase 2A con selección interactiva.

    Args:
        ruta_base_tfm: Ruta base del proyecto TFM

    Returns:
        OrquestadorFase2A con resultados, o None si cancelado
    """
    print("\n" + "="*80)
    print("FASE 2A - EVALUACIÓN CUANTITATIVA Y ANÁLISIS GEOMÉTRICO")
    print("Trabajo Fin de Máster - Universidad Oberta de Catalunya")
    print("="*80 + "\n")

    # Configuración
    config = ConfiguracionFase2A(
        ruta_base_tfm=Path(ruta_base_tfm),
        max_dimension_mahotas=1024,
        ancho_borde_px=5,
        usar_checkpoint=True,
        guardar_json_individuales=True,
        guardar_csv_consolidado=True,
        guardar_estadisticas=True,
        nivel_logging=NivelLogging.NORMAL
    )

    # Cargar índice maestro
    ruta_indice = config.ruta_fase1 / "indice_maestro.json"

    if not ruta_indice.exists():
        raise FileNotFoundError(f"Índice maestro no encontrado: {ruta_indice}")

    with open(ruta_indice, 'r', encoding='utf-8') as f:
        indice_maestro = json.load(f)

    # Selección de modelos
    logger_temp = logging.getLogger('Selector')
    logger_temp.setLevel(logging.INFO)

    selector = SelectorModelos(indice_maestro, logger_temp)

    configuraciones_seleccionadas = selector.seleccionar()

    if configuraciones_seleccionadas is None:
        print("\nEjecución cancelada por el usuario")
        return None

    # Configuración de nivel de logging
    print("\n" + "="*80)
    print("CONFIGURACIÓN DE NIVEL DE LOGGING")
    print("="*80)
    print("\n1. MINIMAL - Solo resumen por foto")
    print("2. NORMAL - Progreso con detalle moderado (recomendado)")
    print("3. DETAILED - Información detallada por configuración")
    print("4. DEBUG - Máximo detalle para debugging")

    while True:
        try:
            nivel_input = input("\nSeleccione nivel [1-4, default=2]: ").strip()

            if nivel_input == '' or nivel_input == '2':
                config.nivel_logging = NivelLogging.NORMAL
                break
            elif nivel_input == '1':
                config.nivel_logging = NivelLogging.MINIMAL
                break
            elif nivel_input == '3':
                config.nivel_logging = NivelLogging.DETAILED
                break
            elif nivel_input == '4':
                config.nivel_logging = NivelLogging.DEBUG
                break
            else:
                print("Por favor ingrese un número entre 1 y 4")

        except KeyboardInterrupt:
            print("\n\nEjecución cancelada por el usuario")
            return None

    # Resumen de configuración
    print(f"\nConfiguración:")
    print(f"  - Ruta base: {config.ruta_base_tfm}")
    print(f"  - Ruta Fase 1: {config.ruta_fase1}")
    print(f"  - Ruta salida: {config.ruta_salida}")
    print(f"  - Configuraciones a procesar: {len(configuraciones_seleccionadas)}")
    print(f"  - Nivel logging: {config.nivel_logging.value}")
    print(f"  - Checkpoint: {config.usar_checkpoint}")
    print()

    # Crear orquestador y ejecutar
    orquestador = OrquestadorFase2A(config, configuraciones_seleccionadas)

    orquestador.ejecutar()

    return orquestador

In [ ]:
# =============================================================================
# PUNTO DE ENTRADA
# =============================================================================

if __name__ == "__main__":

    RUTA_BASE_TFM = "/content/drive/MyDrive/TFM"

    try:
        orquestador = ejecutar_fase2a_interactivo(RUTA_BASE_TFM)

        if orquestador is None:
            print("\nEjecución cancelada")
            sys.exit(0)

        # Resumen final
        print("\n" + "="*80)
        print("RESUMEN DE EJECUCIÓN")
        print("="*80)
        print(f"Fotografías procesadas: {len(orquestador.checkpoint.fotos_procesadas) if orquestador.checkpoint else 'N/A'}")
        print(f"Evaluaciones exitosas: {len(orquestador.resultados_consolidados)}")

        if orquestador.estadisticas_omisiones:
            total_omisiones = sum(len(v) for v in orquestador.estadisticas_omisiones.values())
            print(f"Configuraciones omitidas: {total_omisiones}")

        print(f"\nResultados disponibles en:")
        print(f"  - JSONs individuales: {orquestador.config.ruta_salida / 'metricas_individuales'}")
        print(f"  - CSV consolidado: {orquestador.config.ruta_salida / 'metricas_agregadas' / 'todas_metricas.csv'}")
        print(f"  - Ranking TOP-10: {orquestador.config.ruta_salida / 'metricas_agregadas' / 'ranking_configuraciones.csv'}")
        print(f"  - Estadísticas: {orquestador.config.ruta_salida / 'metricas_agregadas' / 'estadisticas_por_modelo.json'}")
        print(f"  - Reporte omisiones: {orquestador.config.ruta_salida / 'metricas_agregadas' / 'configuraciones_omitidas.json'}")

        print("\n" + "="*80)
        print("FASE 2A COMPLETADA EXITOSAMENTE")
        print("="*80)

    except KeyboardInterrupt:
        print("\n\nEjecución interrumpida por el usuario")
        sys.exit(1)

    except Exception as e:
        print(f"\nERROR DURANTE EJECUCIÓN: {e}")
        import traceback
        traceback.print_exc()
        raise



FASE 2A - EVALUACIÓN CUANTITATIVA Y ANÁLISIS GEOMÉTRICO
Trabajo Fin de Máster - Universidad Oberta de Catalunya


SELECCIÓN DE MODELOS PARA PROCESAMIENTO

Modelos disponibles:

1. BODYPIX
   Configuraciones: 24

2. MASK2FORMER
   Configuraciones: 19

3. ONEFORMER
   Configuraciones: 36

4. SAM2
   Configuraciones: 12

5. SAM2 PROMPTS
   Configuraciones: 32

6. YOLOV8
   Configuraciones: 20

7. TODOS LOS MODELOS
   Total: 143 configuraciones

8. SELECCIÓN MÚLTIPLE (separar con comas)


Seleccione opción [1-8]: 5

RESUMEN DE SELECCIÓN

Total de configuraciones seleccionadas: 32

Primeras 10 configuraciones:
  - sam2_prompts_base_plus_bbox_heuristic
  - sam2_prompts_base_plus_combined_aggressive
  - sam2_prompts_base_plus_combined_moderate
  - sam2_prompts_base_plus_grid_central_aggressive
  - sam2_prompts_base_plus_grid_central_conservative
  - sam2_prompts_base_plus_grid_central_moderate
  - sam2_prompts_base_plus_saliency_conservative
  - sam2_prompts_base_plus_saliency_moderate
  - s

INFO:OrquestadorFase2A:Índice maestro cargado: 20 fotografías


[23:09:34] INFO     | 
Iniciando Fase 2A - Evaluación Cuantitativa


INFO:OrquestadorFase2A:
Iniciando Fase 2A - Evaluación Cuantitativa


[23:09:34] INFO     | Configuraciones a evaluar: 32


...

[01:04:32] INFO     | Reporte de omisiones guardado: /content/drive/MyDrive/TFM/3_Analisis/fase2_evaluacion/metricas_agregadas/configuraciones_omitidas.json


INFO:OrquestadorFase2A:Reporte de omisiones guardado: /content/drive/MyDrive/TFM/3_Analisis/fase2_evaluacion/metricas_agregadas/configuraciones_omitidas.json


[01:04:32] INFO     | 
Fase 2A completada exitosamente


INFO:OrquestadorFase2A:
Fase 2A completada exitosamente



RESUMEN DE EJECUCIÓN
Fotografías procesadas: 20
Evaluaciones exitosas: 588
Configuraciones omitidas: 52

Resultados disponibles en:
  - JSONs individuales: /content/drive/MyDrive/TFM/3_Analisis/fase2_evaluacion/metricas_individuales
  - CSV consolidado: /content/drive/MyDrive/TFM/3_Analisis/fase2_evaluacion/metricas_agregadas/todas_metricas.csv
  - Ranking TOP-10: /content/drive/MyDrive/TFM/3_Analisis/fase2_evaluacion/metricas_agregadas/ranking_configuraciones.csv
  - Estadísticas: /content/drive/MyDrive/TFM/3_Analisis/fase2_evaluacion/metricas_agregadas/estadisticas_por_modelo.json
  - Reporte omisiones: /content/drive/MyDrive/TFM/3_Analisis/fase2_evaluacion/metricas_agregadas/configuraciones_omitidas.json

FASE 2A COMPLETADA EXITOSAMENTE
